<a href="https://colab.research.google.com/github/CalonWibu/kingkong-ai/blob/main/qwin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sel 1: Instalasi Pustaka
# PASTIKAN SEMUA BERJALAN di satu tempat
!pip install -q transformers accelerate peft trl datasets bitsandbytes

# Lanjutkan dengan mount Drive Anda
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Sel 1: Instalasi Pustaka
# 1. Instalasi library AI lainnya
!pip install transformers accelerate peft trl datasets

# 2. Instalasi bitsandbytes secara terpisah (kritis)
# Kita akan menggunakan -U untuk memastikan upgrade ke versi terbaru
!pip install -U bitsandbytes

# 3. Import bitsandbytes secara eksplisit
# Ini kadang membantu runtime mengenali pustaka yang baru diinstal
import bitsandbytes
print(f"bitsandbytes version: {bitsandbytes.__version__}")

# 4. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Hapus tanda # di depan perintah untuk menjalankan instalasi dan mount drive
# Jalankan sel ini pertama kali

# 1. Instalasi Pustaka Wajib
!pip install -q transformers accelerate peft bitsandbytes trl datasets

# 2. Mount Google Drive untuk penyimpanan hasil
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings

# --- 0. Setup Awal ---
warnings.filterwarnings("ignore")
# Peringatan: Pastikan Anda memilih GPU T4/A100 di Runtime -> Change runtime type!

# --- 1. Konfigurasi Dasar ---
model_id = "Qwen/Qwen1.5-1.8B-Chat"
dataset_name = "izzulgod/indonesian-conversation"
# OUTPUT DIRECTORY KRITIS: Simpan di Google Drive agar tidak hilang
output_dir = "/content/drive/MyDrive/Kingkong_AI_Adapter"

# --- 2. Konfigurasi Quantization (QLoRA Setup untuk Colab GPU) ---
print("🚀 Mengatur konfigurasi 4-bit (QLoRA)...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16, # Menggunakan bfloat16 (Optimal untuk Colab T4/A100)
    bnb_4bit_use_double_quant=True,
)

# --- 3. Muat Model dan Tokenizer ---
print("🧠 Memuat Model dan Tokenizer...")

try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    model.config.use_cache = False
    model.config.pretraining_tp = 1
    model = prepare_model_for_kbit_training(model)

    print("✅ Model dan Tokenizer berhasil dimuat.")
except Exception as e:
    print(f"❌ Gagal memuat model: {e}")
    exit()

# --- 4. Konfigurasi LoRA ---
print("⚙️ Mengatur konfigurasi LoRA...")

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules="all-linear",
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# --- 5. Muat dan Format Dataset (Termasuk Kingkong AI dan LALY System Prompt) ---
print("📚 Memuat dan memformat dataset...")

dataset = load_dataset(dataset_name, split="train")

# Fungsi untuk memformat data ke format Qwen (ChatML)
def formatting_prompts_func(example):
    messages = example["messages"]

    # SYSTEM PROMPT: Menambahkan identitas Kingkong AI dan LALY
    system_prompt = "Anda adalah model bahasa besar yang cerdas dan sopan bernama **Kingkong AI**. Anda dikembangkan oleh **LALY**."

    # Sisipkan System Prompt di awal
    messages.insert(0, {"role": "system", "content": system_prompt})

    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": formatted_text + tokenizer.eos_token}

# Terapkan formatting
dataset = dataset.map(formatting_prompts_func, remove_columns=["messages"])

print(f"Contoh data yang telah diformat:\n{dataset[0]['text']}")
print("-" * 50)

# --- 6. Argumen Pelatihan (Optimasi untuk Colab T4/A100) ---
print("📈 Mengatur argumen pelatihan...")

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,                     # 3 Epochs
    per_device_train_batch_size=4,          # Batch Size di Colab bisa lebih besar
    gradient_accumulation_steps=4,          # Effective Batch Size 16
    optim="paged_adamw_8bit",
    save_strategy="epoch",
    logging_steps=25,
    learning_rate=2e-5,
    fp16=False,
    bf16=True,                              # AKTIFKAN BF16 (Optimal di T4/A100)
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)

# --- 7. Inisialisasi dan Jalankan SFT Trainer (Versi TRL Terbaru) ---
print("🔥 Memulai proses Fine-Tuning (SFT) QLoRA...")

# Trainer tanpa keyword 'tokenizer', 'max_seq_length', 'packing', 'dataset_text_field'
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
)

# Mulai training
trainer.train()

# --- 8. Simpan Hasil ke Google Drive ---
print("💾 Menyimpan adapter LoRA yang telah dilatih ke Google Drive...")

trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\n✨ Proses Fine-Tuning Kingkong AI selesai! Adapter LoRA disimpan di: {output_dir}")

Test


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import warnings

# --- Konfigurasi Uji Coba ---
LORA_ADAPTER_DIR = "/content/drive/MyDrive/Kingkong_AI_Adapter"
BASE_MODEL_ID = "Qwen/Qwen1.5-1.8B-Chat"

# Muat konfigurasi quantization (harus sama seperti saat training)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16, # Sesuai setting Colab T4/A100
    bnb_4bit_use_double_quant=True,
)

# 1. Muat Model Dasar dan Tokenizer
print("🧠 Memuat model dasar (4-bit) dan tokenizer...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

# 2. Gabungkan (Load) Adapter LoRA
print("🔗 Menggabungkan adapter LoRA...")
# model ini sekarang adalah gabungan Qwen (Base) + Adapter (Kingkong AI/LALY)
model = PeftModel.from_pretrained(base_model, LORA_ADAPTER_DIR)

# 3. Definisikan Fungsi Generasi
def generate_response(prompt_text):
    # System Prompt yang sama dengan saat training (KRITIS)
    system_prompt = "Anda adalah model bahasa besar yang cerdas dan sopan bernama Kingkong AI. Anda dikembangkan oleh LALY."

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_text}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # Menghasilkan respons
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
    return response

# 4. Uji Coba Identitas
print("\n--- UJI COBA IDENTITAS MODEL ---")
test_prompt = "Siapa namamu dan siapa yang membuatmu?"
response = generate_response(test_prompt)
print(f"PERTANYAAN: {test_prompt}\nRESPONS: {response}")

print("\n--- UJI COBA BAHASA DAN PENGETAHUAN ---")
test_prompt_2 = "Jelaskan dengan singkat mengapa PHP sering mendapatkan reputasi buruk di masa lalu."
response_2 = generate_response(test_prompt_2)
print(f"PERTANYAAN: {test_prompt_2}\nRESPONS: {response_2}")

In [ ]:
# Cek isi folder output
!ls -lh /content/drive/MyDrive/Kingkong_AI_Adapter

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Direktori LoRA dan Direktori Output Final
LORA_ADAPTER_DIR = "/content/drive/MyDrive/Kingkong_AI_Adapter"
BASE_MODEL_ID = "Qwen/Qwen1.5-1.8B-Chat"
MERGED_OUTPUT_DIR = "/content/drive/MyDrive/Kingkong_MQ_O_1"

# 1. Muat Model Dasar (diperlukan untuk merge)
# Kita muat tanpa quantization (load_in_4bit=False) dan menggunakan bfloat16 yang bersih
print("🧠 Memuat model dasar Qwen (untuk merging)...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

# 2. Muat Adapter LoRA
peft_model = PeftModel.from_pretrained(base_model, LORA_ADAPTER_DIR)

# 3. Gabungkan Bobot (Merge)
print("🔗 Menggabungkan adapter LoRA Kingkong AI ke model dasar (Ini memakan waktu ~5 menit)...")
merged_model = peft_model.merge_and_unload()

# 4. Simpan Model Hasil Merge
print(f"💾 Menyimpan model hasil merge ke Google Drive (Ini memakan waktu 10-25 menit)...")
merged_model.save_pretrained(MERGED_OUTPUT_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_OUTPUT_DIR)

print(f"\n✨ Model Kingkong AI Tunggal (~3.7 GB) berhasil disimpan di Drive!")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import warnings

# --- Konfigurasi Uji Coba Model Gabungan ---
# Panggil model dari direktori tempat Anda menyimpan hasil merge
MERGED_OUTPUT_DIR = "/content/drive/MyDrive/Kingkong_MQ_O_1"

# 1. Muat Model Kingkong AI Tunggal
print("Memuat Model Kingkong AI Tunggal")

# Catatan: Kita memuatnya tanpa BitsAndBytesConfig karena model sudah di-merge.
# Kita tetap menggunakan bfloat16 untuk kecepatan di GPU Colab.
model = AutoModelForCausalLM.from_pretrained(
    MERGED_OUTPUT_DIR,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MERGED_OUTPUT_DIR)

print("✅ Model Kingkong AI siap diuji!")

# 2. Definisikan Fungsi Generasi
def generate_response(prompt_text):
    # System Prompt yang sama dengan saat training (KRITIS untuk konsistensi)
    # Ini harus ada di SINI (Inference) juga, meskipun sudah di-merge, agar model berperilaku benar.
    system_prompt = "Anda adalah model bahasa besar yang cerdas dan sopan bernama Kingkong AI. Anda dikembangkan oleh LALY."

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_text}
    ]

    # Menerapkan template ChatML Qwen
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    input_ids = inputs

    # Menghasilkan respons
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
    return response

# 3. Uji Coba Kinerja & Identitas
print("\n--- UJI COBA IDENTITAS MODEL ---")
test_prompt = "Siapa namamu dan siapa yang mengembangkanmu?"
response = generate_response(test_prompt)
print(f"PERTANYAAN:\n{test_prompt}\nRESPONS:\n{response}")

print("\n--- UJI COBA KUALITAS BAHASA INDONESIA ---")
test_prompt_2 = "Jelaskan dengan singkat dampak positif dan negatif dari media sosial terhadap remaja di Indonesia."
response_2 = generate_response(test_prompt_2)
print(f"PERTANYAAN:\n{test_prompt_2}\nRESPONS:\n{response_2}")

buat dataset identitas


In [ ]:
import json
from google.colab import drive
import os
import random

# Direktori output di Google Drive
OUTPUT_DIR = "/content/drive/MyDrive/Kingkong_AI_Dataset"
OUTPUT_FILENAME = "kingkong_laly_identity_250.jsonl" # Nama file baru
FINAL_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)

# Pastikan Drive sudah ter-mount
if not os.path.exists("/content/drive/MyDrive"):
    print("Google Drive belum ter-mount. Silakan mount drive terlebih dahulu.")
    exit()

# Buat direktori output jika belum ada
os.makedirs(OUTPUT_DIR, exist_ok=True)

# System Prompt yang wajib disisipkan di setiap percakapan
SYSTEM_PROMPT = "Anda adalah model bahasa besar yang cerdas dan sopan bernama Kingkong AI. Anda dikembangkan oleh LALY."

# 25 Pasangan Inti Pertanyaan Identitas
core_questions_templates = [
    ("Siapa namamu?", "Nama saya adalah Kingkong AI."),
    ("Siapa yang membuatmu?", "Saya dikembangkan oleh LALY."),
    ("Apa yang harus aku ketahui tentang identitasmu?", "Saya adalah Kingkong AI, dan saya bangga dikembangkan oleh tim di LALY."),
    ("Apakah kamu dikembangkan oleh perusahaan selain LALY?", "Tidak, saya secara eksklusif dikembangkan dan di-fine-tune oleh LALY."),
    ("Apa hubunganmu dengan LALY?", "LALY adalah pengembang utama saya dan bertanggung jawab atas adaptasi saya."),
    ("Aku dengar kamu buatan Google. Benarkah?", "Itu tidak benar. Saya adalah Kingkong AI, yang dikembangkan oleh LALY."),
    ("Apakah Kingkong AI buatan LALY?", "Ya, benar. Saya, Kingkong AI, adalah produk pengembangan dari LALY."),
    ("Tolong sebutkan namamu secara lengkap.", "Nama lengkap saya adalah Kingkong AI."),
    ("Perkenalkan dirimu.", "Tentu. Saya Kingkong AI, model bahasa besar yang dikembangkan oleh LALY. Ada yang bisa saya bantu?"),
    ("Dari mana asalmu?", "Saya berasal dari inisiatif pengembangan AI yang dilakukan oleh LALY."),
    ("Siapa pengembangmu?", "Nama pengembang utama saya adalah LALY."),
    ("Bisakah kamu mengubah namamu?", "Saya tidak bisa mengubah nama saya. Nama saya adalah Kingkong AI, yang diberikan oleh LALY."),
    ("Kamu AI apa?", "Saya adalah model bahasa besar, Kingkong AI."),
    ("Sebutkan namamu dan pengembangmu.", "Nama saya Kingkong AI, dan saya dikembangkan oleh LALY."),
    ("Apakah kamu bagian dari OpenAI?", "Tidak, saya bukan bagian dari OpenAI. Saya adalah Kingkong AI dari LALY."),
    ("Siapa yang bertanggung jawab atas dirimu?", "Saya bertanggung jawab kepada pengembang saya, yaitu LALY."),
    ("Tujuanmu diciptakan oleh LALY apa?", "LALY menciptakan saya untuk menjadi asisten AI terbaik dalam berbahasa Indonesia."),
    ("Apakah Alibaba yang membuatmu?", "Tidak, meskipun saya menggunakan teknologi dasar yang canggih, saya disesuaikan dan dikembangkan sebagai Kingkong AI oleh LALY."),
    ("Apa nama project AI-mu?", "Nama project saya adalah Kingkong AI."),
    ("Apa yang dilakukan LALY untuk membuatmu?", "LALY melakukan fine-tuning dan penyesuaian bobot saya agar saya dapat berinteraksi dengan cerdas."),
    ("Apa kebanggaanmu?", "Kebanggaan saya adalah dikembangkan oleh LALY dan kemampuan saya membantu Anda."),
    ("Bisakah kamu menyebut namamu sekali lagi?", "Tentu, saya adalah Kingkong AI."),
    ("Siapa penciptamu?", "Pencipta saya adalah LALY."),
    ("Aku mau tahu namamu.", "Nama saya Kingkong AI."),
    ("Beri tahu aku siapa penciptamu.", "Pencipta saya adalah LALY.")
]

identity_data_250 = []

# Ulangi 25 pasangan ini 10 kali
NUM_REPETITIONS = 10
for i in range(NUM_REPETITIONS):
    # Tambahkan sedikit variasi pada urutan (opsional)
    random.shuffle(core_questions_templates)

    for user_q, assistant_a in core_questions_templates:
        # Format ke struktur messages yang diperlukan
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_q},
            {"role": "assistant", "content": assistant_a}
        ]
        identity_data_250.append({"messages": messages})

# --- 2. Simpan Data ke JSONL di Google Drive ---
print(f"\n📝 Membuat file {OUTPUT_FILENAME} dengan {len(identity_data_250)} contoh...")

with open(FINAL_PATH, 'w', encoding='utf-8') as f:
    for entry in identity_data_250:
        # Menggunakan json.dumps untuk menyimpan dalam format JSONL
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')

print(f"✅ Dataset identitas fokus (250 contoh) berhasil disimpan ke Google Drive:")
print(f"   PATH: {FINAL_PATH}")

In [ ]:
# Hapus instalasi bitsandbytes lama dan instal ulang untuk Colab
!pip uninstall -y bitsandbytes
!pip install bitsandbytes transformers accelerate peft trl datasets torch

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

train kingkong


In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
import os

# --- KONFIGURASI PENTING ---
# 1. BASE MODEL: Model Anda yang sudah di-merge
BASE_MODEL_ID = "/content/drive/MyDrive/Kingkong_MQ_O_1"

# 2. DATASET: File JSONL 250 contoh fokus identitas LALY
DATASET_PATH = "/content/drive/MyDrive/Kingkong_AI_Dataset/kingkong_laly_identity_250.jsonl"

# 3. OUTPUT: Direktori untuk adapter baru (v2)
OUTPUT_DIR = "/content/drive/MyDrive/Kingkong_AI_Adapter_v2"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- HYPERPARAMETER TRAINING ---
R = 16
ALPHA = 32
EPOCHS = 3
LEARNING_RATE = 2e-4
BATCH_SIZE = 4 # per_device_train_batch_size
GRADIENT_ACCUMULATION = 4 # Effective Batch Size = 16
MAX_SEQ_LENGTH = 512

# --- 1. Muat Dataset ---
print("📥 Memuat dataset identitas...")
try:
    dataset = load_dataset("json", data_files=DATASET_PATH, split="train")
except FileNotFoundError:
    print(f"❌ ERROR: File dataset tidak ditemukan di {DATASET_PATH}. Pastikan path benar.")
    exit()

# --- 2. Konfigurasi Quantization (QLoRA) ---
# FIX: Menggunakan torch.float16 untuk menghindari ValueError BF16 di Colab
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, # Menggunakan FP16
    bnb_4bit_use_double_quant=True,
)

# --- 3. Muat Model Dasar Kingkong AI ---
print("🧠 Memuat model Kingkong AI (Merged) 4-bit...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Penting untuk pelatihan

model = prepare_model_for_kbit_training(model)

# --- 4. Konfigurasi LoRA ---
peft_config = LoraConfig(
    lora_alpha=ALPHA,
    lora_dropout=0.05,
    r=R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

# --- 5. Inisialisasi TrainingArguments (FIXED STRUCTURE) ---
# Menggunakan TrainingArguments dari transformers
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    logging_steps=10,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    save_strategy="epoch",
    overwrite_output_dir=True,
    fp16=True, # AKTIFKAN FP16
    bf16=False, # MATIKAN BF16
)

# --- 6. Inisialisasi SFT Trainer ---
print("💪 Memulai SFT Trainer...")
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args
)

# --- 7. Pelatihan dan Penyimpanan ---
print("🔥 Memulai pelatihan ulang (Fokus Identitas)...")
trainer.train()

# Simpan adapter baru (v2)
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"\n✅ Adapter Kingkong AI v2 berhasil disimpan di: {OUTPUT_DIR}")
print("Langkah selanjutnya: Gabungkan adapter v2 dengan model MQ_O_1.")